In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
# import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime
from bs4 import BeautifulSoup
from sklearn.model_selection import GridSearchCV


In [3]:
df = pd.read_csv(r'F:\mpst_full_data.csv', delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 14828 rows and 6 columns


In [4]:
df.head(5)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import preprocessing

Using TensorFlow backend.


In [6]:
# df=df.drop(columns=["split"])

In [7]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split',
       'synopsis_source'],
      dtype='object')

In [8]:
# df['combined']=df['title']+". "+ df['plot_synopsis']

In [9]:
type(Sequential()) == Sequential


True

In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import plotly.plotly as py
# import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot

In [11]:
def gdd(x):
    a=x.split()
    return len(a)

In [12]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [14]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in tqdm(df['plot_synopsis'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())

100%|███████████████████████████████████████████████████████████████████████████| 14828/14828 [00:29<00:00, 510.85it/s]


In [15]:
df['preprocessed_plots']=preprocessed_synopsis

In [16]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split', 'synopsis_source',
       'preprocessed_plots'],
      dtype='object')

In [17]:
def remove_spaces(x):
    x=x.split(",")
    nospace=[]
    for item in x:
        item=item.lstrip()
        nospace.append(item)
    return (",").join(nospace)  

In [18]:
df['tags']=df['tags'].apply(remove_spaces)

In [19]:
train=df.loc[df.split=='train']
cv=df.loc[df.split=="val"]
cv=cv.reset_index()
train=train.reset_index()
test=df.loc[df.split=='test']
test=test.reset_index()

In [20]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['tags']).toarray()
y_test=vectorizer.transform(test['tags']).toarray()
y_cv=vectorizer.transform(cv['tags']).toarray()

In [21]:
import tensorflow as tf

session_conf = tf.ConfigProto(
 intra_op_parallelism_threads=1, 
                        inter_op_parallelism_threads=1, 
                      )

In [22]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split', 'synopsis_source',
       'preprocessed_plots'],
      dtype='object')

In [23]:
max(df['preprocessed_plots'].apply(gdd))

6044

In [24]:
vect=Tokenizer()
vect.fit_on_texts(train['preprocessed_plots'])
vocab_size = len(vect.word_index) + 1

In [25]:
vocab_size

98782

In [26]:
encoded_docs_train = vect.texts_to_sequences(train['preprocessed_plots'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1200, padding='post')
print(padded_docs_train)

[[  680  4577 54887 ...     0     0     0]
 [    6  4356    69 ...     0     0     0]
 [ 2953   323   108 ...    19    66  6797]
 ...
 [   66  2554   401 ...     0     0     0]
 [ 4799  2605  2908 ...     0     0     0]
 [ 1160  2299  2425 ...     0     0     0]]


In [27]:
encoded_docs_test = vect.texts_to_sequences(test['preprocessed_plots'])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200, padding='post')
encoded_docs_cv = vect.texts_to_sequences(cv['preprocessed_plots'])
padded_docs_cv = pad_sequences(encoded_docs_cv, maxlen=1200, padding='post')



In [47]:
session_conf = tf.ConfigProto(
 intra_op_parallelism_threads=1, 
                        inter_op_parallelism_threads=1, 
                      )

In [28]:
def f1micro(y_true, y_pred):
    return tf.py_func(f1_score(y_true, y_pred,average='mirco'),tf.double)


In [29]:
# embedding_matrix.shape

In [30]:
# embedding_matrix.shape[1]

In [31]:
# timesteps = len(x_train[0])
# input_dim = len(x_train[0])
# # n_classes = _count_classes(y_train)
# n_classes=147
# print(timesteps)
# print(input_dim)
# print(len(x_train[0]))

In [32]:
# model_glove = Sequential()
# model_glove.add(Embedding(vocab_size,output_dim=embedding_matrix.shape[1],input_length=1200,weights=[embedding_matrix]))
# model_glove.add(LSTM(256, return_sequences=True,activation='relu'))
# model_glove.add(Flatten())
# model_glove.add(Dense(256, activation='relu'))
# model_glove.add(Dropout(0.5))
# model_glove.add(Dense(256, activation='relu'))
# model_glove.add(Dropout(0.5))
# model_glove.add(Dense(71, activation='sigmoid'))
# model_glove.compile(loss='binary_crossentropy', optimizer='adam')
# ## Fit train data
# model_glove.fit(padded_docs, y_train,validation_data=(padded_docs_cv,y_cv), epochs =5,batch_size=32)

In [33]:
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import Adam
from time import time
from keras.callbacks import TensorBoard
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam

from keras.layers.embeddings import Embedding
from keras.callbacks import ReduceLROnPlateau

In [63]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')

In [39]:
# class Metrics(Callback):
    
#     def on_train_begin(self, logs={}):
#         self.val_f1s = []

#         self.val_recalls = []
#         self.val_precisions=[]
#     #     sel.val_precisions = []

#     #     sef.val_precisions = []

#     def on_epoch_end(self, epoch, logs={}):

#         val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
#         val_targ = self.model.validation_data[1]
#         _val_f1 = f1_score(val_targ, val_predict)
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
#         self.val_f1s.append(_val_f1)
#         self.val_recalls.append(_val_recall)
#         self.val_precisions.append(_val_precision)
# #         print“ — val_f1: %f — val_precision: %f — val_recall %f” %(_val_f1, _val_precision, _val_recall)
#         print( (_al_f1: %f — val_precision: %f — val_recall %f) %(_val_f1, _val_precision, _val_recall))
#         return
 
# metrics = Metrics()

In [41]:
# class Metrics(Callback):
    
#     def on_train_begin(self, logs={}):
#         self.val_f1s = []

#         self.val_recalls = []
#         self.val_precisions=[]
#     #     sel.val_precisions = []

#     #     sef.val_precisions = []

#     def on_epoch_end(self, epoch, logs={}):

#         val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
#         val_targ = self.model.validation_data[1]
#         _val_f1 = f1_score(val_targ, val_predict)
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
#         self.val_f1s.append(_val_f1)
#         self.val_recalls.append(_val_recall)
#         self.val_precisions.append(_val_precision)
# #         print“ — val_f1: %f — val_precision: %f — val_recall %f” %(_val_f1, _val_precision, _val_recall)
#         print(“ — val_f1: %f — val_precision: %f — val_recall %f” %(_val_f1, _val_precision, _val_recall))
# #         return
 

# metrics = Metrics()

In [64]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(71, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1200, 50)          4939100   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1200, 50)          0         
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 71)                3621      
Total params: 4,942,721
Trainable params: 4,942,721
Non-trainable params: 0
_________________________________________________________________


In [65]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1micro])

model.compile(loss='binary_crossentropy', optimizer='adam')
# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-simple1.hdf5', save_best_only=True)
# ]

history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)
# history=model.fit(padded_docs_train, y_train,validation_data=(padded_docs_cv,y_cv), epochs =5,batch_size=32,callbacks=[tensorboard])

Train on 8540 samples, validate on 949 samples
Epoch 1/5
8540/8540 [==============================] - ETA: 5:35 - loss: 0.694 - ETA: 3:12 - loss: 0.693 - ETA: 2:31 - loss: 0.692 - ETA: 2:07 - loss: 0.691 - ETA: 1:54 - loss: 0.690 - ETA: 1:45 - loss: 0.689 - ETA: 1:37 - loss: 0.688 - ETA: 1:31 - loss: 0.687 - ETA: 1:27 - loss: 0.686 - ETA: 1:22 - loss: 0.685 - ETA: 1:19 - loss: 0.684 - ETA: 1:17 - loss: 0.684 - ETA: 1:15 - loss: 0.683 - ETA: 1:13 - loss: 0.682 - ETA: 1:11 - loss: 0.680 - ETA: 1:10 - loss: 0.679 - ETA: 1:09 - loss: 0.678 - ETA: 1:08 - loss: 0.677 - ETA: 1:06 - loss: 0.676 - ETA: 1:05 - loss: 0.675 - ETA: 1:04 - loss: 0.674 - ETA: 1:03 - loss: 0.673 - ETA: 1:02 - loss: 0.672 - ETA: 1:02 - loss: 0.671 - ETA: 1:01 - loss: 0.670 - ETA: 1:00 - loss: 0.669 - ETA: 59s - loss: 0.668 - ETA: 59s - loss: 0.66 - ETA: 58s - loss: 0.66 - ETA: 57s - loss: 0.66 - ETA: 57s - loss: 0.66 - ETA: 56s - loss: 0.66 - ETA: 56s - loss: 0.66 - ETA: 55s - loss: 0.65 - ETA: 55s - loss: 0.65 - ETA: 

8540/8540 [==============================] - ETA: 46s - loss: 0.13 - ETA: 52s - loss: 0.15 - ETA: 53s - loss: 0.15 - ETA: 53s - loss: 0.15 - ETA: 54s - loss: 0.15 - ETA: 53s - loss: 0.16 - ETA: 54s - loss: 0.16 - ETA: 55s - loss: 0.16 - ETA: 54s - loss: 0.16 - ETA: 53s - loss: 0.16 - ETA: 54s - loss: 0.15 - ETA: 54s - loss: 0.16 - ETA: 54s - loss: 0.16 - ETA: 54s - loss: 0.16 - ETA: 53s - loss: 0.16 - ETA: 53s - loss: 0.16 - ETA: 53s - loss: 0.16 - ETA: 52s - loss: 0.15 - ETA: 52s - loss: 0.15 - ETA: 52s - loss: 0.15 - ETA: 51s - loss: 0.15 - ETA: 51s - loss: 0.15 - ETA: 51s - loss: 0.15 - ETA: 50s - loss: 0.15 - ETA: 50s - loss: 0.15 - ETA: 50s - loss: 0.15 - ETA: 50s - loss: 0.15 - ETA: 49s - loss: 0.15 - ETA: 49s - loss: 0.15 - ETA: 49s - loss: 0.15 - ETA: 48s - loss: 0.15 - ETA: 48s - loss: 0.15 - ETA: 48s - loss: 0.15 - ETA: 48s - loss: 0.15 - ETA: 47s - loss: 0.15 - ETA: 47s - loss: 0.15 - ETA: 47s - loss: 0.15 - ETA: 47s - loss: 0.15 - ETA: 46s - loss: 0.15 - ETA: 46s - loss: 0.

8540/8540 [==============================] - ETA: 51s - loss: 0.14 - ETA: 55s - loss: 0.12 - ETA: 57s - loss: 0.13 - ETA: 57s - loss: 0.13 - ETA: 58s - loss: 0.14 - ETA: 58s - loss: 0.13 - ETA: 58s - loss: 0.13 - ETA: 56s - loss: 0.13 - ETA: 55s - loss: 0.13 - ETA: 54s - loss: 0.13 - ETA: 54s - loss: 0.13 - ETA: 53s - loss: 0.13 - ETA: 53s - loss: 0.13 - ETA: 53s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.

8540/8540 [==============================] - ETA: 48s - loss: 0.15 - ETA: 52s - loss: 0.15 - ETA: 52s - loss: 0.15 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 51s - loss: 0.13 - ETA: 51s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 53s - loss: 0.14 - ETA: 53s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.

8540/8540 [==============================] - ETA: 51s - loss: 0.12 - ETA: 50s - loss: 0.14 - ETA: 52s - loss: 0.15 - ETA: 51s - loss: 0.15 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.15 - ETA: 50s - loss: 0.15 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 45s - loss: 0.14 - ETA: 45s - loss: 0.14 - ETA: 45s - loss: 0.14 - ETA: 45s - loss: 0.14 - ETA: 44s - loss: 0.14 - ETA: 44s - loss: 0.14 - ETA: 44s - loss: 0.

In [67]:
filter_length = 300


# model = Sequential()
# model.add(Embedding(6119, 71, input_length=1200))
# model.add(Dropout(0.1))
# model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
# model.add(GlobalMaxPool1D())
# model.add(Dense(82))
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(71, activation='sigmoid'))

# model.summary()


model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-2.h5', save_best_only=True)
# ]
history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1200, 50)          4939100   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1200, 50)          0         
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 50)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 71)                3621      
_________________________________________________________________
activation_3 (Activation)    (None, 71)                0         
Total params: 4,942,721
Trainable params: 4,942,721
Non-trainable params: 0
_________________________________________________________________
Train on 8540 samples, validate on 949 samples
Epoch 1/5
8540/8540 [==============================] - ETA: 5:48 - loss: 0.953 - ETA:

8540/8540 [==============================] - ETA: 51s - loss: 0.70 - ETA: 50s - loss: 0.70 - ETA: 50s - loss: 0.70 - ETA: 51s - loss: 0.70 - ETA: 52s - loss: 0.70 - ETA: 51s - loss: 0.70 - ETA: 51s - loss: 0.70 - ETA: 51s - loss: 0.70 - ETA: 50s - loss: 0.70 - ETA: 50s - loss: 0.70 - ETA: 50s - loss: 0.70 - ETA: 50s - loss: 0.70 - ETA: 49s - loss: 0.70 - ETA: 49s - loss: 0.70 - ETA: 49s - loss: 0.70 - ETA: 49s - loss: 0.70 - ETA: 49s - loss: 0.70 - ETA: 48s - loss: 0.70 - ETA: 48s - loss: 0.70 - ETA: 48s - loss: 0.70 - ETA: 48s - loss: 0.70 - ETA: 48s - loss: 0.70 - ETA: 47s - loss: 0.70 - ETA: 47s - loss: 0.70 - ETA: 47s - loss: 0.70 - ETA: 47s - loss: 0.70 - ETA: 47s - loss: 0.70 - ETA: 46s - loss: 0.70 - ETA: 46s - loss: 0.70 - ETA: 46s - loss: 0.70 - ETA: 46s - loss: 0.70 - ETA: 46s - loss: 0.70 - ETA: 45s - loss: 0.70 - ETA: 45s - loss: 0.70 - ETA: 45s - loss: 0.70 - ETA: 45s - loss: 0.70 - ETA: 45s - loss: 0.70 - ETA: 44s - loss: 0.70 - ETA: 44s - loss: 0.70 - ETA: 44s - loss: 0.

8540/8540 [==============================] - ETA: 51s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 44s - loss: 0.69 - ETA: 44s - loss: 0.

8540/8540 [==============================] - ETA: 48s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 44s - loss: 0.69 - ETA: 44s - loss: 0.

8540/8540 [==============================] - ETA: 51s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 46s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 45s - loss: 0.69 - ETA: 44s - loss: 0.69 - ETA: 44s - loss: 0.69 - ETA: 44s - loss: 0.69 - ETA: 44s - loss: 0.

In [69]:
# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-3.h5', save_best_only=True)
# ]
# history = model.fit(padded_docs_train, y_train,
#                     class_weight='balanced',
#                     epochs=5,
#                     batch_size=32,
#                     validation_split=0.1,
#                     callbacks=callbacks)

In [73]:
# filter_length = 300
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Dropout(0.5))
model.add(Conv1D(300, 3, activation='relu'))
model.add(GlobalMaxPool1D())
# model.add(Conv1D(200, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
model.add(Dense(71))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1200, 71)          7013522   
_________________________________________________________________
dropout_11 (Dropout)         (None, 1200, 71)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1198, 300)         64200     
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 71)                21371     
_________________________________________________________________
activation_5 (Activation)    (None, 71)                0         
Total params: 7,099,093
Trainable params: 7,099,093
Non-trainable params: 0
_________________________________________________________________


In [74]:
# model.fit(x_train, y_train, batch_size=16, epochs=10)
# log_path=str('C:\Users\Cyborg\Documents\GitHub\Movie-Plot-Synopses')
# model_saver =  ModelCheckpoint(log_path + "/model.ckpt.{epoch:04d}.hdf5", monitor='loss', period=1)
# callbacks = [tensorboard, TerminateOnNaN(), model_saver]


model.fit(padded_docs_train, y_train,
          class_weight='balanced',
          epochs=12,
          batch_size=32,
          validation_split=0.2)
predictions=model.predict(padded_docs_test)

Train on 7591 samples, validate on 1898 samples
Epoch 1/12
7591/7591 [==============================] - ETA: 9:55 - loss: 0.696 - ETA: 6:43 - loss: 0.691 - ETA: 5:34 - loss: 0.685 - ETA: 4:58 - loss: 0.679 - ETA: 4:36 - loss: 0.672 - ETA: 4:23 - loss: 0.666 - ETA: 4:12 - loss: 0.659 - ETA: 4:02 - loss: 0.652 - ETA: 3:56 - loss: 0.645 - ETA: 3:51 - loss: 0.637 - ETA: 3:47 - loss: 0.630 - ETA: 3:45 - loss: 0.622 - ETA: 3:43 - loss: 0.614 - ETA: 3:44 - loss: 0.606 - ETA: 3:42 - loss: 0.597 - ETA: 3:39 - loss: 0.589 - ETA: 3:36 - loss: 0.580 - ETA: 3:33 - loss: 0.571 - ETA: 3:31 - loss: 0.561 - ETA: 3:29 - loss: 0.551 - ETA: 3:26 - loss: 0.541 - ETA: 3:26 - loss: 0.532 - ETA: 3:25 - loss: 0.522 - ETA: 3:24 - loss: 0.512 - ETA: 3:23 - loss: 0.502 - ETA: 3:22 - loss: 0.493 - ETA: 3:20 - loss: 0.482 - ETA: 3:18 - loss: 0.473 - ETA: 3:16 - loss: 0.464 - ETA: 3:15 - loss: 0.455 - ETA: 3:13 - loss: 0.446 - ETA: 3:11 - loss: 0.437 - ETA: 3:10 - loss: 0.429 - ETA: 3:08 - loss: 0.422 - ETA: 3:07 - 

7591/7591 [==============================] - ETA: 3:19 - loss: 0.172 - ETA: 3:47 - loss: 0.164 - ETA: 3:36 - loss: 0.147 - ETA: 3:31 - loss: 0.151 - ETA: 3:27 - loss: 0.144 - ETA: 3:25 - loss: 0.141 - ETA: 3:22 - loss: 0.141 - ETA: 3:21 - loss: 0.141 - ETA: 3:19 - loss: 0.142 - ETA: 3:17 - loss: 0.142 - ETA: 3:15 - loss: 0.144 - ETA: 3:14 - loss: 0.145 - ETA: 3:13 - loss: 0.143 - ETA: 3:11 - loss: 0.143 - ETA: 3:10 - loss: 0.141 - ETA: 3:09 - loss: 0.142 - ETA: 3:08 - loss: 0.142 - ETA: 3:07 - loss: 0.142 - ETA: 3:06 - loss: 0.142 - ETA: 3:05 - loss: 0.143 - ETA: 3:04 - loss: 0.144 - ETA: 3:03 - loss: 0.144 - ETA: 3:02 - loss: 0.144 - ETA: 3:01 - loss: 0.144 - ETA: 3:00 - loss: 0.143 - ETA: 2:59 - loss: 0.143 - ETA: 2:58 - loss: 0.143 - ETA: 2:57 - loss: 0.144 - ETA: 2:57 - loss: 0.144 - ETA: 2:56 - loss: 0.143 - ETA: 2:55 - loss: 0.142 - ETA: 2:54 - loss: 0.143 - ETA: 2:53 - loss: 0.142 - ETA: 2:52 - loss: 0.143 - ETA: 2:51 - loss: 0.142 - ETA: 2:50 - loss: 0.142 - ETA: 2:49 - loss: 0

7591/7591 [==============================] - ETA: 3:18 - loss: 0.162 - ETA: 3:18 - loss: 0.151 - ETA: 3:17 - loss: 0.160 - ETA: 3:15 - loss: 0.152 - ETA: 3:14 - loss: 0.152 - ETA: 3:14 - loss: 0.156 - ETA: 3:13 - loss: 0.152 - ETA: 3:12 - loss: 0.150 - ETA: 3:11 - loss: 0.147 - ETA: 3:10 - loss: 0.146 - ETA: 3:10 - loss: 0.144 - ETA: 3:08 - loss: 0.144 - ETA: 3:08 - loss: 0.145 - ETA: 3:07 - loss: 0.145 - ETA: 3:06 - loss: 0.148 - ETA: 3:05 - loss: 0.147 - ETA: 3:04 - loss: 0.146 - ETA: 3:03 - loss: 0.145 - ETA: 3:03 - loss: 0.144 - ETA: 3:02 - loss: 0.144 - ETA: 3:01 - loss: 0.144 - ETA: 3:00 - loss: 0.144 - ETA: 2:59 - loss: 0.144 - ETA: 2:58 - loss: 0.144 - ETA: 2:57 - loss: 0.145 - ETA: 2:57 - loss: 0.145 - ETA: 2:56 - loss: 0.144 - ETA: 2:55 - loss: 0.143 - ETA: 2:54 - loss: 0.142 - ETA: 2:53 - loss: 0.143 - ETA: 2:52 - loss: 0.143 - ETA: 2:51 - loss: 0.144 - ETA: 2:51 - loss: 0.144 - ETA: 2:50 - loss: 0.144 - ETA: 2:49 - loss: 0.144 - ETA: 2:48 - loss: 0.144 - ETA: 2:48 - loss: 0

7591/7591 [==============================] - ETA: 3:21 - loss: 0.117 - ETA: 3:18 - loss: 0.121 - ETA: 3:16 - loss: 0.136 - ETA: 3:15 - loss: 0.136 - ETA: 3:14 - loss: 0.135 - ETA: 3:13 - loss: 0.139 - ETA: 3:12 - loss: 0.139 - ETA: 3:12 - loss: 0.141 - ETA: 3:11 - loss: 0.146 - ETA: 3:10 - loss: 0.145 - ETA: 3:09 - loss: 0.144 - ETA: 3:09 - loss: 0.145 - ETA: 3:08 - loss: 0.143 - ETA: 3:07 - loss: 0.144 - ETA: 3:07 - loss: 0.144 - ETA: 3:06 - loss: 0.143 - ETA: 3:05 - loss: 0.143 - ETA: 3:05 - loss: 0.143 - ETA: 3:04 - loss: 0.144 - ETA: 3:03 - loss: 0.142 - ETA: 3:02 - loss: 0.140 - ETA: 3:01 - loss: 0.140 - ETA: 3:00 - loss: 0.139 - ETA: 3:00 - loss: 0.138 - ETA: 2:59 - loss: 0.138 - ETA: 2:58 - loss: 0.138 - ETA: 2:57 - loss: 0.138 - ETA: 2:56 - loss: 0.138 - ETA: 2:55 - loss: 0.138 - ETA: 2:54 - loss: 0.139 - ETA: 2:54 - loss: 0.139 - ETA: 2:53 - loss: 0.139 - ETA: 2:52 - loss: 0.139 - ETA: 2:51 - loss: 0.139 - ETA: 2:50 - loss: 0.139 - ETA: 2:49 - loss: 0.139 - ETA: 2:48 - loss: 0

7591/7591 [==============================] - ETA: 3:16 - loss: 0.127 - ETA: 3:16 - loss: 0.123 - ETA: 3:15 - loss: 0.120 - ETA: 3:13 - loss: 0.118 - ETA: 3:12 - loss: 0.124 - ETA: 3:12 - loss: 0.121 - ETA: 3:11 - loss: 0.124 - ETA: 3:11 - loss: 0.126 - ETA: 3:10 - loss: 0.128 - ETA: 3:09 - loss: 0.127 - ETA: 3:08 - loss: 0.127 - ETA: 3:07 - loss: 0.129 - ETA: 3:06 - loss: 0.130 - ETA: 3:05 - loss: 0.129 - ETA: 3:05 - loss: 0.130 - ETA: 3:04 - loss: 0.130 - ETA: 3:03 - loss: 0.128 - ETA: 3:02 - loss: 0.128 - ETA: 3:01 - loss: 0.131 - ETA: 3:00 - loss: 0.130 - ETA: 2:59 - loss: 0.131 - ETA: 2:59 - loss: 0.132 - ETA: 2:58 - loss: 0.131 - ETA: 2:57 - loss: 0.131 - ETA: 2:56 - loss: 0.131 - ETA: 2:55 - loss: 0.132 - ETA: 2:55 - loss: 0.132 - ETA: 2:54 - loss: 0.131 - ETA: 2:53 - loss: 0.131 - ETA: 2:52 - loss: 0.131 - ETA: 2:51 - loss: 0.132 - ETA: 2:51 - loss: 0.132 - ETA: 2:50 - loss: 0.132 - ETA: 2:49 - loss: 0.133 - ETA: 2:48 - loss: 0.133 - ETA: 2:47 - loss: 0.133 - ETA: 2:47 - loss: 0

...

7591/7591 [==============================] - ETA: 3:18 - loss: 0.125 - ETA: 3:17 - loss: 0.124 - ETA: 3:16 - loss: 0.121 - ETA: 3:14 - loss: 0.120 - ETA: 3:14 - loss: 0.120 - ETA: 3:13 - loss: 0.120 - ETA: 3:13 - loss: 0.122 - ETA: 3:12 - loss: 0.123 - ETA: 3:11 - loss: 0.124 - ETA: 3:10 - loss: 0.124 - ETA: 3:09 - loss: 0.124 - ETA: 3:08 - loss: 0.123 - ETA: 3:07 - loss: 0.122 - ETA: 3:06 - loss: 0.122 - ETA: 3:06 - loss: 0.123 - ETA: 3:06 - loss: 0.123 - ETA: 3:05 - loss: 0.122 - ETA: 3:04 - loss: 0.122 - ETA: 3:03 - loss: 0.122 - ETA: 3:02 - loss: 0.121 - ETA: 3:02 - loss: 0.121 - ETA: 3:01 - loss: 0.120 - ETA: 3:00 - loss: 0.119 - ETA: 2:59 - loss: 0.119 - ETA: 2:58 - loss: 0.117 - ETA: 2:57 - loss: 0.116 - ETA: 2:56 - loss: 0.117 - ETA: 2:56 - loss: 0.116 - ETA: 2:55 - loss: 0.116 - ETA: 2:54 - loss: 0.117 - ETA: 2:53 - loss: 0.117 - ETA: 2:52 - loss: 0.117 - ETA: 2:51 - loss: 0.117 - ETA: 2:50 - loss: 0.118 - ETA: 2:49 - loss: 0.118 - ETA: 2:48 - loss: 0.118 - ETA: 2:48 - loss: 0

7591/7591 [==============================] - ETA: 3:19 - loss: 0.119 - ETA: 3:17 - loss: 0.119 - ETA: 3:15 - loss: 0.115 - ETA: 3:14 - loss: 0.112 - ETA: 3:14 - loss: 0.115 - ETA: 3:13 - loss: 0.115 - ETA: 3:12 - loss: 0.114 - ETA: 3:11 - loss: 0.114 - ETA: 3:10 - loss: 0.116 - ETA: 3:09 - loss: 0.115 - ETA: 3:08 - loss: 0.115 - ETA: 3:08 - loss: 0.116 - ETA: 3:07 - loss: 0.115 - ETA: 3:06 - loss: 0.115 - ETA: 3:05 - loss: 0.114 - ETA: 3:04 - loss: 0.114 - ETA: 3:03 - loss: 0.116 - ETA: 3:03 - loss: 0.116 - ETA: 3:02 - loss: 0.116 - ETA: 3:01 - loss: 0.116 - ETA: 3:00 - loss: 0.115 - ETA: 2:59 - loss: 0.115 - ETA: 2:59 - loss: 0.116 - ETA: 2:58 - loss: 0.116 - ETA: 2:57 - loss: 0.117 - ETA: 2:56 - loss: 0.117 - ETA: 2:55 - loss: 0.116 - ETA: 2:54 - loss: 0.117 - ETA: 2:54 - loss: 0.117 - ETA: 2:53 - loss: 0.117 - ETA: 2:52 - loss: 0.117 - ETA: 2:51 - loss: 0.117 - ETA: 2:50 - loss: 0.117 - ETA: 2:49 - loss: 0.116 - ETA: 2:49 - loss: 0.116 - ETA: 2:48 - loss: 0.116 - ETA: 2:47 - loss: 0

7591/7591 [==============================] - ETA: 3:18 - loss: 0.120 - ETA: 3:19 - loss: 0.112 - ETA: 3:18 - loss: 0.109 - ETA: 3:16 - loss: 0.107 - ETA: 3:15 - loss: 0.106 - ETA: 3:14 - loss: 0.111 - ETA: 3:13 - loss: 0.110 - ETA: 3:13 - loss: 0.110 - ETA: 3:12 - loss: 0.109 - ETA: 3:11 - loss: 0.110 - ETA: 3:10 - loss: 0.111 - ETA: 3:09 - loss: 0.111 - ETA: 3:08 - loss: 0.110 - ETA: 3:06 - loss: 0.109 - ETA: 3:06 - loss: 0.109 - ETA: 3:05 - loss: 0.109 - ETA: 3:04 - loss: 0.109 - ETA: 3:03 - loss: 0.111 - ETA: 3:02 - loss: 0.112 - ETA: 3:01 - loss: 0.112 - ETA: 3:01 - loss: 0.113 - ETA: 3:00 - loss: 0.114 - ETA: 2:59 - loss: 0.113 - ETA: 2:58 - loss: 0.113 - ETA: 2:57 - loss: 0.112 - ETA: 2:57 - loss: 0.113 - ETA: 2:56 - loss: 0.112 - ETA: 2:55 - loss: 0.113 - ETA: 2:54 - loss: 0.112 - ETA: 2:53 - loss: 0.113 - ETA: 2:52 - loss: 0.114 - ETA: 2:51 - loss: 0.113 - ETA: 2:50 - loss: 0.113 - ETA: 2:50 - loss: 0.113 - ETA: 2:49 - loss: 0.113 - ETA: 2:48 - loss: 0.113 - ETA: 2:47 - loss: 0

7591/7591 [==============================] - ETA: 3:17 - loss: 0.101 - ETA: 3:18 - loss: 0.104 - ETA: 3:17 - loss: 0.098 - ETA: 3:16 - loss: 0.096 - ETA: 3:15 - loss: 0.099 - ETA: 3:14 - loss: 0.101 - ETA: 3:13 - loss: 0.106 - ETA: 3:12 - loss: 0.106 - ETA: 3:12 - loss: 0.103 - ETA: 3:11 - loss: 0.105 - ETA: 3:10 - loss: 0.105 - ETA: 3:09 - loss: 0.104 - ETA: 3:08 - loss: 0.103 - ETA: 3:07 - loss: 0.103 - ETA: 3:06 - loss: 0.102 - ETA: 3:05 - loss: 0.101 - ETA: 3:04 - loss: 0.101 - ETA: 3:03 - loss: 0.101 - ETA: 3:02 - loss: 0.101 - ETA: 3:01 - loss: 0.102 - ETA: 3:01 - loss: 0.102 - ETA: 3:00 - loss: 0.102 - ETA: 2:59 - loss: 0.102 - ETA: 2:58 - loss: 0.102 - ETA: 2:57 - loss: 0.102 - ETA: 2:56 - loss: 0.103 - ETA: 2:55 - loss: 0.104 - ETA: 2:54 - loss: 0.104 - ETA: 2:54 - loss: 0.105 - ETA: 2:53 - loss: 0.105 - ETA: 2:52 - loss: 0.106 - ETA: 2:51 - loss: 0.105 - ETA: 2:50 - loss: 0.105 - ETA: 2:50 - loss: 0.105 - ETA: 2:49 - loss: 0.105 - ETA: 2:48 - loss: 0.105 - ETA: 2:47 - loss: 0

7591/7591 [==============================] - ETA: 5:17 - loss: 0.154 - ETA: 5:14 - loss: 0.121 - ETA: 5:15 - loss: 0.105 - ETA: 5:13 - loss: 0.102 - ETA: 5:12 - loss: 0.101 - ETA: 5:11 - loss: 0.099 - ETA: 5:10 - loss: 0.097 - ETA: 5:08 - loss: 0.096 - ETA: 5:06 - loss: 0.097 - ETA: 5:05 - loss: 0.095 - ETA: 5:04 - loss: 0.096 - ETA: 5:02 - loss: 0.096 - ETA: 5:01 - loss: 0.096 - ETA: 5:00 - loss: 0.095 - ETA: 4:58 - loss: 0.095 - ETA: 4:57 - loss: 0.095 - ETA: 4:56 - loss: 0.094 - ETA: 4:54 - loss: 0.093 - ETA: 4:53 - loss: 0.096 - ETA: 4:52 - loss: 0.096 - ETA: 4:50 - loss: 0.098 - ETA: 4:49 - loss: 0.098 - ETA: 4:47 - loss: 0.099 - ETA: 4:46 - loss: 0.099 - ETA: 4:45 - loss: 0.099 - ETA: 4:44 - loss: 0.098 - ETA: 4:43 - loss: 0.098 - ETA: 4:41 - loss: 0.098 - ETA: 4:40 - loss: 0.098 - ETA: 4:38 - loss: 0.098 - ETA: 4:37 - loss: 0.097 - ETA: 4:36 - loss: 0.099 - ETA: 4:34 - loss: 0.099 - ETA: 4:33 - loss: 0.098 - ETA: 4:31 - loss: 0.098 - ETA: 4:30 - loss: 0.098 - ETA: 4:29 - loss: 0

In [ ]:
n_classes=71

In [54]:
# score = model.evaluate(x_test, y_test, batch_size=16)

In [75]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# Configuring the parameters
# model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(LSTM(128))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()
# pred=[]
# x= model.predict(y_test)
# for i in x:
#     pred.append(i[:])  
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 1200, 50)          4939100   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 71)                9159      
Total params: 5,039,907
Trainable params: 5,039,907
Non-trainable params: 0
_________________________________________________________________


In [76]:
# model = Sequential()
# # model.add(Embedding(max_words, 20, input_length=maxlen))
# model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(LSTM(128))
# model.add(Dropout(0.15))
# model.add(GlobalMaxPool1D())
# model.add(Dense(71, activation='sigmoid'))

# # model.summary()


# model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-2.h5', save_best_only=True)
# ]
history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 1200, 50)          4939100   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 71)                9159      
Total params: 5,039,907
Trainable params: 5,039,907
Non-trainable params: 0
_________________________________________________________________
Train on 8540 samples, validate on 949 samples
Epoch 1/5
8540/8540 [==============================] - ETA: 26:08 - loss: 0.69 - ETA: 18:47 - loss: 0.69 - ETA: 16:27 - loss: 0.69 - ETA: 15:14 - loss: 0.68 - ETA: 14:29 - loss: 0.68 - ETA: 13:58 - loss: 0.68 - ETA: 1

8540/8540 [==============================] - ETA: 6:05 - loss: 0.134 - ETA: 6:07 - loss: 0.160 - ETA: 6:06 - loss: 0.152 - ETA: 6:06 - loss: 0.150 - ETA: 6:04 - loss: 0.153 - ETA: 6:03 - loss: 0.154 - ETA: 6:01 - loss: 0.151 - ETA: 6:00 - loss: 0.149 - ETA: 5:58 - loss: 0.151 - ETA: 5:56 - loss: 0.149 - ETA: 5:56 - loss: 0.149 - ETA: 5:54 - loss: 0.149 - ETA: 5:52 - loss: 0.151 - ETA: 5:51 - loss: 0.151 - ETA: 5:49 - loss: 0.149 - ETA: 5:48 - loss: 0.149 - ETA: 5:46 - loss: 0.149 - ETA: 5:45 - loss: 0.150 - ETA: 5:43 - loss: 0.150 - ETA: 5:42 - loss: 0.152 - ETA: 5:41 - loss: 0.151 - ETA: 5:39 - loss: 0.152 - ETA: 5:38 - loss: 0.153 - ETA: 5:36 - loss: 0.152 - ETA: 5:35 - loss: 0.150 - ETA: 5:34 - loss: 0.150 - ETA: 5:32 - loss: 0.150 - ETA: 5:31 - loss: 0.150 - ETA: 5:29 - loss: 0.151 - ETA: 5:28 - loss: 0.151 - ETA: 5:27 - loss: 0.150 - ETA: 5:25 - loss: 0.151 - ETA: 5:24 - loss: 0.150 - ETA: 5:22 - loss: 0.150 - ETA: 5:21 - loss: 0.150 - ETA: 5:20 - loss: 0.151 - ETA: 5:18 - loss: 0

8540/8540 [==============================] - ETA: 6:10 - loss: 0.141 - ETA: 6:07 - loss: 0.139 - ETA: 6:04 - loss: 0.139 - ETA: 6:03 - loss: 0.150 - ETA: 6:02 - loss: 0.156 - ETA: 6:01 - loss: 0.151 - ETA: 6:00 - loss: 0.146 - ETA: 5:58 - loss: 0.145 - ETA: 5:57 - loss: 0.143 - ETA: 5:56 - loss: 0.145 - ETA: 5:55 - loss: 0.144 - ETA: 5:53 - loss: 0.147 - ETA: 5:51 - loss: 0.147 - ETA: 5:50 - loss: 0.145 - ETA: 5:49 - loss: 0.145 - ETA: 5:48 - loss: 0.145 - ETA: 5:46 - loss: 0.145 - ETA: 5:45 - loss: 0.146 - ETA: 5:43 - loss: 0.145 - ETA: 5:42 - loss: 0.144 - ETA: 5:40 - loss: 0.144 - ETA: 5:39 - loss: 0.143 - ETA: 5:38 - loss: 0.144 - ETA: 5:36 - loss: 0.143 - ETA: 5:35 - loss: 0.143 - ETA: 5:33 - loss: 0.143 - ETA: 5:32 - loss: 0.144 - ETA: 5:31 - loss: 0.145 - ETA: 5:30 - loss: 0.145 - ETA: 5:28 - loss: 0.144 - ETA: 5:27 - loss: 0.143 - ETA: 5:26 - loss: 0.143 - ETA: 5:24 - loss: 0.145 - ETA: 5:23 - loss: 0.145 - ETA: 5:21 - loss: 0.144 - ETA: 5:20 - loss: 0.145 - ETA: 5:19 - loss: 0

8540/8540 [==============================] - ETA: 6:05 - loss: 0.150 - ETA: 6:03 - loss: 0.147 - ETA: 6:06 - loss: 0.152 - ETA: 6:05 - loss: 0.150 - ETA: 6:03 - loss: 0.151 - ETA: 6:01 - loss: 0.148 - ETA: 6:00 - loss: 0.144 - ETA: 5:58 - loss: 0.142 - ETA: 5:57 - loss: 0.144 - ETA: 5:56 - loss: 0.145 - ETA: 5:55 - loss: 0.142 - ETA: 5:53 - loss: 0.145 - ETA: 5:52 - loss: 0.147 - ETA: 5:51 - loss: 0.145 - ETA: 5:50 - loss: 0.145 - ETA: 5:48 - loss: 0.144 - ETA: 5:47 - loss: 0.144 - ETA: 5:46 - loss: 0.143 - ETA: 5:44 - loss: 0.145 - ETA: 5:43 - loss: 0.145 - ETA: 5:41 - loss: 0.145 - ETA: 5:40 - loss: 0.146 - ETA: 5:39 - loss: 0.147 - ETA: 5:37 - loss: 0.146 - ETA: 5:36 - loss: 0.146 - ETA: 5:35 - loss: 0.147 - ETA: 5:33 - loss: 0.146 - ETA: 5:32 - loss: 0.146 - ETA: 5:30 - loss: 0.145 - ETA: 5:29 - loss: 0.145 - ETA: 5:27 - loss: 0.145 - ETA: 5:26 - loss: 0.145 - ETA: 5:25 - loss: 0.145 - ETA: 5:23 - loss: 0.145 - ETA: 5:22 - loss: 0.144 - ETA: 5:20 - loss: 0.144 - ETA: 5:19 - loss: 0

8540/8540 [==============================] - ETA: 6:15 - loss: 0.144 - ETA: 6:13 - loss: 0.156 - ETA: 6:09 - loss: 0.150 - ETA: 6:06 - loss: 0.140 - ETA: 6:05 - loss: 0.140 - ETA: 6:03 - loss: 0.136 - ETA: 6:01 - loss: 0.136 - ETA: 6:00 - loss: 0.136 - ETA: 5:59 - loss: 0.139 - ETA: 5:57 - loss: 0.142 - ETA: 5:56 - loss: 0.148 - ETA: 5:54 - loss: 0.148 - ETA: 5:53 - loss: 0.148 - ETA: 5:51 - loss: 0.147 - ETA: 5:50 - loss: 0.146 - ETA: 5:49 - loss: 0.145 - ETA: 5:47 - loss: 0.146 - ETA: 5:45 - loss: 0.144 - ETA: 5:44 - loss: 0.145 - ETA: 5:43 - loss: 0.145 - ETA: 5:42 - loss: 0.145 - ETA: 5:40 - loss: 0.145 - ETA: 5:39 - loss: 0.145 - ETA: 5:37 - loss: 0.146 - ETA: 5:36 - loss: 0.146 - ETA: 5:35 - loss: 0.147 - ETA: 5:33 - loss: 0.146 - ETA: 5:32 - loss: 0.145 - ETA: 5:31 - loss: 0.145 - ETA: 5:29 - loss: 0.145 - ETA: 5:28 - loss: 0.145 - ETA: 5:26 - loss: 0.144 - ETA: 5:25 - loss: 0.144 - ETA: 5:23 - loss: 0.143 - ETA: 5:22 - loss: 0.144 - ETA: 5:20 - loss: 0.144 - ETA: 5:19 - loss: 0

In [ ]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# Configuring the parameters
# model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(LSTM(128))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

In [ ]:
# model.compile(optimizer='adam', loss='binary_crossentropy')
# model.summary()

# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-6.h5', save_best_only=True)
# ]
# history = model.fit(padded_docs_train, y_train,
#                     class_weight='balanced',
#                     epochs=5,
#                     batch_size=32,
#                     validation_split=0.1,
#                     callbacks=[])

In [59]:
# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-4.h5', save_best_only=True)
# ]
# history = model.fit(padded_docs_train, y_train,
#                     class_weight='balanced',
#                     epochs=5,
#                     batch_size=32,
#                     validation_split=0.1,
#                     callbacks=[])

In [ ]:
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_test, pred)

In [ ]:
# model.predict(y_test[0]).shape

In [ ]:
# model = Sequential()
# model.add(Embedding(vocab_size, output_dim=50))
# # model.add(LSTM(300))
# # model.add(LSTM(256, return_sequences=True))
# model.add(LSTM(128))
# model.add(Dropout(0.5))
# # model.add(LSTM(128))

# # model.add(LSTM(150))

# # model.add(Dropout(0.5))
# model.add(Dense(71, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.summary()

In [ ]:
# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])


In [ ]:
# model.fit(padded_docs_train, y_train,
#           class_weight='balanced',
#           epochs=12,
#           batch_size=32,
#           validation_split=0.2)